In [47]:
!pip install python-dotenv groq pandas --quiet

In [75]:
from dotenv import load_dotenv
from groq import Groq
import os
import requests
import urllib.request

In [8]:
load_dotenv()

GROQ_API_KEY = os.getenv('GROQ_API_KEY')

## Reading parsed data

In [48]:
import pandas as pd

In [51]:
integrations_list = pd.read_json(path_or_buf="blogs.jsonl", lines=True)

integrations_list

,name,link,integrations_found
0,4 ways RevOps can elevate customer experiences...,https://zapier.com/blog/how-revenue-operations...,None
1,5 ways to automate tl;dv with Zapier,https://zapier.com/blog/automate-tldv/,[[Create pages in Notion for new tl;dv recordi...
2,5 ways to automate Coda with Zapier,https://zapier.com/blog/automate-coda/,[[Update and organize rows in Coda whenever ne...
3,Your guide to Zoho automation,https://zapier.com/blog/zoho-automation-guide/,[[Create contacts on Zoho CRM from new leads f...
4,4 ways to automate Podio with Zapier,https://zapier.com/blog/automate-podio/,[[Save Gravity Forms submissions as Podio item...
...,...,...,...
498,Popular ways to use Zapier to improve your work,https://zapier.com/blog/popular-zaps/,"[[Post new RSS items to Facebook Pages, RSS by..."
499,Close More Sales with Copper: 5 Teams Share Th...,https://zapier.com/blog/prosperworks-automations/,[[Add new Gravity Forms submissions as leads i...
500,How 5 Teams Use MeisterTask and Zapier to Boos...,https://zapier.com/blog/meistertask-integrations/,[[Create Google Calendar events from new Meist...
501,How to Use Google Apps Script,https://zapier.com/blog/google-apps-script-tut...,[[Create Google Contacts from new Google Sheet...


Let's pick a particular task from one of the parsed articles:

In [65]:
article = integrations_list.iloc[3]

print(article['name'])
print(article['link'])
print()

'''
for ind, i in enumerate(article['integrations_found']):
    print(ind, i)
'''

#article['integrations_found'][6]
task_example = article['integrations_found'][13]
task_example

Your guide to Zoho automation
https://zapier.com/blog/zoho-automation-guide/



['Send Gmail emails for new Zoho Mail emails', 'Zoho Mail + Gmail']

## Zero-shot learning

In [67]:
client = Groq(api_key=GROQ_API_KEY)

def generate(task, apis, lang, temperature=0.0, max_tokens=1024):
    res = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"""
                You are a {lang} expert. 
                You generate integrations that allow users to access APIs. 
                No explanations.
                Make sure that you use the APIs that the user lists.
                Double-check the correctness of your code.
                """,
            },
            {
                "role": "user",
                "content": f'''
                Generate a script that can {task}. 
                You will be using the following APIs: {apis}.
                ''',
            }
        ],
        model="llama3-70b-8192", #llama3-8b-8192
        temperature=temperature,
        max_tokens=max_tokens,
        #response_format= {"type": "json_object"}

        # IDEA: the LLM can return a JSON object with:
        # 1. The code for the integration
        # 2. The list of constants to be replaced (stuff to obtain: credentials, API keys, etc.)
    )
    return res.choices[0].message.content

In [68]:
print(generate(task_example[0], task_example[1], 'TypeScript'))

Here is a TypeScript script that integrates Zoho Mail and Gmail APIs to send Gmail emails for new Zoho Mail emails:
```
import * as axios from 'axios';
import * as google from 'googleapis';

// Zoho Mail API credentials
const zohoMailClientId = 'YOUR_ZOHO_MAIL_CLIENT_ID';
const zohoMailClientSecret = 'YOUR_ZOHO_MAIL_CLIENT_SECRET';
const zohoMailRefreshToken = 'YOUR_ZOHO_MAIL_REFRESH_TOKEN';

// Gmail API credentials
const gmailClientId = 'YOUR_GMAIL_CLIENT_ID';
const gmailClientSecret = 'YOUR_GMAIL_CLIENT_SECRET';
const gmailRefreshToken = 'YOUR_GMAIL_REFRESH_TOKEN';

// Set up Zoho Mail API
const zohoMailAuth = {
  client_id: zohoMailClientId,
  client_secret: zohoMailClientSecret,
  refresh_token: zohoMailRefreshToken,
};

const zohoMailApi = axios.create({
  baseURL: 'https://www.zoho.com/mail/api/',
  headers: {
    'Authorization': `Bearer ${getZohoMailAccessToken()}`,
  },
});

// Set up Gmail API
const gmailAuth = new google.auth.GoogleAuth({
  client_id: gmailClientId,
  clien

## Few-shot learning

The idea here is to find relevant integrations in Windmill Community Integrations.


The task description and the corresponding source code is added to the context window of the LLM as a reference to ensure valid generated code.

In [92]:
source_code_url = 'https://raw.githubusercontent.com/windmill-labs/windmill-community-integrations/main/integrations/mailgun/Send_Email/script.bun.ts'

url = source_code_url
r = requests.get(url).text
print(os.linesep.join(r.split(os.linesep)[:20]))
print('...')

type Mailgun = {
	apiKey: string
	domain: string
}

export async function main(
	resource: Mailgun,
	data: {
		from: string
		to: string
		subject: string
		html: string
		recipient_variables?: string
		template?: string
		cc?: string
		bcc?: string
		text?: string
		amp_html?: string
		attachment?: string
		inline?: string
...


In [95]:
def generate_few_shot(task, apis, lang, temperature=0.0, max_tokens=1024):
    res = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": f"""
                You are a {lang} expert. 
                You generate integrations that allow users to access APIs. 
                No explanations.
                Make sure that you use the APIs that the user lists.
                Double-check the correctness of your code.
                Use this as an example: {r}.
                Remember: your code has to follow the same style. Do not deviate from this format.
                """,
            },
            {
                "role": "user",
                "content": f'''
                Generate a script that can {task}. 
                You will be using the following APIs: {apis}.
                ''',
            }
        ],
        model="llama3-70b-8192", #llama3-8b-8192
        temperature=temperature,
        max_tokens=max_tokens,
        #response_format= {"type": "json_object"}
    )
    return res.choices[0].message.content

In [96]:
print(generate_few_shot(task_example[0], task_example[1], 'TypeScript'))

Here is a script that integrates Zoho Mail and Gmail APIs to send Gmail emails for new Zoho Mail emails:

```typescript
type ZohoMail = {
  clientId: string
  clientSecret: string
  refreshToken: string
  redirectUri: string
}

type Gmail = {
  clientId: string
  clientSecret: string
  refreshToken: string
  redirectUri: string
}

export async function main(
  zohoMailResource: ZohoMail,
  gmailResource: Gmail,
  data: {
    from: string
    to: string
    subject: string
    body: string
  }
) {
  // Get access token for Zoho Mail API
  const zohoMailAccessToken = await getAccessToken(zohoMailResource)

  // Get new emails from Zoho Mail API
  const newEmailsResponse = await fetch(`https://www.zoho.com/mail/api/accounts/${zohoMailResource.clientId}/messages?folder=INBOX&label=UNREAD`, {
    method: 'GET',
    headers: {
      Authorization: `Bearer ${zohoMailAccessToken}`
    }
  })

  const newEmails = await newEmailsResponse.json()

  // Get access token for Gmail API
  const gmailA

## TODO

* ### use Perplexity AI to check whether an OpenAPI spec exists. If it does, we don't need to use LLMs to generate code.
* ### use LLMs to generate tests for each integration
* ### can we separate an integration that uses multiple services into a sequence of integrations that depend on one service only? Does it make sense?